In [1]:
!python make_dataset.py
!python train_model.py

[{'len': 990, 'name': '<cascade.data.pickler.Pickler', 'type': 'dataset'},
 {'len': 990,
  'n_features': 2,
  'name': '<cascade.data.range_sampler.RangeSampler',
  'type': 'dataset'},
 {'name': '<utils.utils.MakeClassificationDataset', 'type': 'dataset'}]
[{'len': 10, 'name': '<cascade.data.pickler.Pickler', 'type': 'dataset'},
 {'len': 10,
  'name': '<cascade.data.range_sampler.RangeSampler',
  'type': 'dataset'},
 {'name': '<utils.utils.MakeClassificationDataset', 'type': 'dataset'}]


The system cannot find the path specified.


[{'name': '<cascade.utils.sk_model.SkModel object at 0x0000025E2202E9A0>', 'created_at': DateTime(2022, 11, 30, 20, 53, 49, 239304, tzinfo=Timezone('UTC')), 'metrics': {'f1': 0.9644268774703558}, 'params': {}, 'type': 'model', 'pipeline': "Pipeline(steps=[('0', SVC(probability=True))])"}]


The system cannot find the path specified.


In [2]:
from cascade.models import ModelRepo
from cascade.meta import MetricViewer
from cascade import data as cdd

In [3]:
from correctness import correctness
from continuity import continuity
from contrastivity import contrastivity
from coherence import coherence

c:\xaib-experiments\venv3.8\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from xaib.explainers.feature_importance.constant_explainer import ConstantExplainer
from xaib.explainers.feature_importance.random_explainer import RandomExplainer
from xaib.explainers.feature_importance.shap_explainer import ShapExplainer
from xaib.explainers.feature_importance.lime_explainer import LimeExplainer

In [5]:
BS = 20

In [6]:
train_ds = cdd.Pickler('train_ds')

In [7]:
explainers = {    
    'const': ConstantExplainer(n_features=2, constant=1),
    'random': RandomExplainer(n_features=2),
    'shap': ShapExplainer(train_ds),
    'lime': LimeExplainer(train_ds, labels=(0, 1))
}

In [8]:
correctness(explainers, batch_size=BS)

100%|██████████| 1/1 [00:00<?, ?it/s]
Exact explainer: 11it [00:11,  1.14s/it]
100%|██████████| 1/1 [00:02<00:00,  2.27s/it]


In [9]:
continuity(explainers, batch_size=BS)

1it [00:00, 443.37it/s]
1it [00:00, 1000.55it/s]
1it [00:00,  2.82it/s]
1it [00:03,  3.75s/it]


In [10]:
contrastivity(explainers, batch_size=BS)

100%|██████████| 1/1 [00:00<00:00,  1.45it/s]


In [11]:
coherence(explainers, batch_size=BS)

100%|██████████| 1/1 [00:02<00:00,  2.37s/it]


In [12]:
repo = ModelRepo('repo')

In [18]:
t = MetricViewer(repo).table

In [29]:
def present_metrics(case):
    return t[t['line'].apply(lambda x: case in x)].dropna(axis=1)

In [30]:
present_metrics('coherence')

,line,num,created_at,saved,other_disagreement,name
0,repo\coherence,0,2022-11-30 20:54:23.368819+00:00,a few seconds after,1.123431,const
1,repo\coherence,1,2022-11-30 20:54:23.368819+00:00,a few seconds after,0.725261,random
2,repo\coherence,2,2022-11-30 20:54:23.368819+00:00,a few seconds after,1.020178,shap
3,repo\coherence,3,2022-11-30 20:54:23.368819+00:00,a few seconds after,0.776460,lime


In [31]:
present_metrics('continuity')

,line,num,created_at,saved,name,small_noise_check,multiplier
4,repo\continuity,0,2022-11-30 20:54:17.380209+00:00,a few seconds after,const,0.000000,0.01
5,repo\continuity,1,2022-11-30 20:54:17.380209+00:00,a few seconds after,random,0.505191,0.01
6,repo\continuity,2,2022-11-30 20:54:17.380209+00:00,a few seconds after,shap,0.000000,0.01
7,repo\continuity,3,2022-11-30 20:54:17.380209+00:00,a few seconds after,lime,0.012557,0.01


In [34]:
present_metrics('contrastivity')

,line,num,created_at,saved,name,label_difference
8,repo\contrastivity,0,2022-11-30 20:54:21.642265+00:00,a few seconds after,const,0.000000
9,repo\contrastivity,1,2022-11-30 20:54:21.642265+00:00,a few seconds after,random,0.406890
10,repo\contrastivity,2,2022-11-30 20:54:21.642265+00:00,a few seconds after,shap,0.000000
11,repo\contrastivity,3,2022-11-30 20:54:21.642265+00:00,a few seconds after,lime,0.013779


In [35]:
present_metrics('correctness')

,line,num,created_at,saved,parameter_randomization_check
12,repo\correctness,0,2022-11-30 20:54:03.135343+00:00,a few seconds after,0.000000
13,repo\correctness,1,2022-11-30 20:54:03.135343+00:00,a few seconds after,0.562691
14,repo\correctness,2,2022-11-30 20:54:03.135343+00:00,11 seconds after,0.480565
15,repo\correctness,3,2022-11-30 20:54:03.135343+00:00,14 seconds after,0.630902
